In [ ]:
!pip install qiskit

In [ ]:
#Import packages

import numpy as np
import pandas as pd
from PIL import Image
import os
import path

import torch
import torchvision
from torch.utils import data
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import qiskit
from qiskit.visualization import *
from qiskit import ClassicalRegister

import random

In [ ]:
# def target_to_oh(target):
#     NUM_CLASS = 5  # hard code here, can do partial
#     one_hot = torch.eye(NUM_CLASS)[target]
#     return one_hot

In [ ]:
transform_bunch = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomPerspective(),
                                      transforms.RandomRotation(15),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5139, 0.2727, 0.0891), (0.1533, 0.0909, 0.0400))])

In [ ]:
ds = torchvision.datasets.ImageFolder("../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images", transform_bunch)#, target_transform = target_to_oh)

In [ ]:
for x,y in ds:
    print(y)
    break

In [ ]:
percent = 0.2

split = int(len(ds) * percent)

indices = list(range(len(ds)))

In [ ]:
indices

In [ ]:
random.shuffle(indices)

In [ ]:
indices

In [ ]:
train_sampler = SubsetRandomSampler(indices[split:])

valid_sampler = SubsetRandomSampler(indices[:split])

In [ ]:
bs = 1

dl = data.DataLoader(ds, bs, sampler=train_sampler)

valid_dl = data.DataLoader(ds, bs, sampler=valid_sampler)

In [ ]:
xx = 0
yy = 0

for x,y in valid_dl:
    print(x.shape,y.shape)
    xx = x
    yy = y
    break

In [ ]:
yy

In [ ]:
#looks like something is off... Yup --> We can just transform it at the end
ds.class_to_idx

In [ ]:
QC_outputs = ['000', '001', '010', '011', '100', '101', '110', '111']

In [ ]:
# Defines the quantum circuit so we can use it (since hybrids have quantum)
class QuantumCircuit:
  #This is the initialization
  def __init__(self, n_qubits, backend, shots):
    #Define how many lanes we want
    self._circuit = qiskit.QuantumCircuit(n_qubits)

    #Just a list of 0 to the number of qubits... Just useful so we can just define the circuit (with all it's little parts super quickly)
    all_qubits = [i for i in range(n_qubits)]
    #There are 7 placeholder variables
    self.theta_0 = qiskit.circuit.Parameter('theta0')
    self.theta_1 = qiskit.circuit.Parameter('theta1')
    self.theta_2 = qiskit.circuit.Parameter('theta2')
    self.theta_3 = qiskit.circuit.Parameter('theta3')
    self.theta_4 = qiskit.circuit.Parameter('theta4')
    self.theta_5 = qiskit.circuit.Parameter('theta5')
    self.theta_6 = qiskit.circuit.Parameter('theta6')

    #Shove in the Hardav gate, a barrier (visual), and a rotation about the y plane of theta degrees
    self._circuit.h(all_qubits)
    self._circuit.barrier()
    self._circuit.ry(self.theta_0, all_qubits)
    #Now comes the custom thing for 3 qubits specifically. I adapted it from a paper
    self._circuit.cz(0,1)
    self._circuit.cz(1,2)
    self._circuit.ry(self.theta_1, 0)
    self._circuit.ry(self.theta_2, 1)
    self._circuit.ry(self.theta_3, 2)
    self._circuit.cz(0,1)
    self._circuit.cz(1,2)
    self._circuit.ry(self.theta_4, 0)
    self._circuit.ry(self.theta_5, 1)
    self._circuit.ry(self.theta_6, 2)

    self._circuit.measure_all()

    #save these varaibles for later so we don't have to call them again during the forwarding
    self.backend = backend
    self.shots = shots
  
  #forwarding through the quantum circuit
  def run(self, thetas):
    #prep the execution. Link to circuit, Define backend and number of shots... And then fill in the placeholder variables (theta) with the thing we pass through when we forward
    job = qiskit.execute(self._circuit,
                         self.backend,
                         shots = self.shots,
                         parameter_binds = [{self.theta_0: thetas[0],
                                             self.theta_1: thetas[1],
                                             self.theta_2: thetas[2],
                                             self.theta_3: thetas[3],
                                             self.theta_4: thetas[4],
                                             self.theta_5: thetas[5],
                                             self.theta_6: thetas[6],}])
    
    #execution
    counts = job.result().get_counts(self._circuit)

    expects = np.zeros(8)
    for k in range(8):
      key = QC_outputs[k]
      perc = counts.get(key, 0) /self.shots
      expects[k] = perc
    return expects

In [ ]:
#Going to test out the quantum circuit.
sim = qiskit.Aer.get_backend('qasm_simulator')

test_circuit = QuantumCircuit(3, sim, 100000)

In [ ]:
#Heck yeah it works! It outputs the distributions of the outputs
test_circuit.run([1,2,1,4,5,6,8]), test_circuit._circuit.draw()

In [ ]:
#This class defines what our hybrid layer does. It allows it to go forward, and also backprop
class HybridFunction(Function):

  @staticmethod
  def forward(ctx, input, quantum_circuit, shift):
    ctx.shift = shift
    ctx.quantum_circuit = quantum_circuit

    parameter = []

    #A very scrapy but functional way of loading the parameters in this format
    for x in input:
      for y in x:
        parameter.append(float(y))
    # print(parameter)

    #Aka we run the input into our circuit
    expectation_z = ctx.quantum_circuit.run(parameter) #Might need to change this when have multi-layer
    #Shoves to pytorch tesnor
    result = torch.tensor([expectation_z])
    #Save the input and result for backpropagation
    ctx.save_for_backward(input,result)
    #return output
    return result
  
  @staticmethod
  def backward(ctx, grad_output):
    #Regrabing the input and the output
    input, expectation_z = ctx.saved_tensors
    # input_list = np.array(input.tolist())
    # print(input)

    input_list = list(input)[0]
    # print(input_list)

    input_list = input_list.cpu()

    #A list of all the gradients
    gradients = torch.Tensor()

    #We're going to go through the inputs and then calculate the gradient for each one
    for i in range(7):
      #In order to get the gradients for each one, we shift one of the parameters a tiny bit and find the difference in the outputs... Since we can't do backprop on it, we use something called shift parameter
      shift_right = np.array(input_list.detach().clone())
      shift_left = np.array(input_list.detach().clone())
      shift_right[i] = shift_right[i] + ctx.shift
      shift_left[i] = shift_left[i] - ctx.shift

      #So we take the shifted ones and just compute it 
      expectation_right = ctx.quantum_circuit.run(shift_right)
      expectation_left  = ctx.quantum_circuit.run(shift_left)
      #Gradient = appox the difference (division by 2*shift isn't necessary since it'll just be a scaled version (which can be counteracted by lr))
      gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
      #Append the gradient to the meta list
      gradients  = torch.cat((gradients, gradient.float()))
    
    #Format everything
    result = torch.Tensor(gradients)
    result = result.float()
    result = result.T

    #Just a random varaible that made my life easier when debugging
    Fixer = grad_output.float()
    Fixer = Fixer.T

    #Find the gradients at last!!!
    h = result * Fixer
    h = h.cuda()
    #then return it for backprop
    return h, None, None

In [ ]:
#This defines the acutal thing that we're shoving into the NN
class Hybrid(nn.Module):
  #initialization
  def __init__(self, backend, shots, shift):
    super(Hybrid, self).__init__()
    #Define the real quantum circuit that we'll be using for our thing
    self.quantum_circuit = QuantumCircuit(3, backend, shots)
    #Save this guy for alter
    self.shift = shift
  
  #When forwarding
  def forward(self, input):
    return HybridFunction.apply(input, self.quantum_circuit, self.shift)

In [ ]:
#Now I'll be making the model! It'll take elements off of nn.Module
class cnnmaker(nn.Module):
    #Initialization will only take in input channels
    def __init__(self, input_channels):
        super().__init__()
        
        #I looked at what the resnet architechture looked like and just implmented the block types (not the residual part just yet)
        def block(in_chan):
            return nn.Sequential(nn.Conv2d(in_chan, in_chan * 3, 3, 2, 1, padding_mode = "reflect"),
                                 nn.MaxPool2d(2),
                                nn.ReLU())
        
        self.model = nn.Sequential(block(input_channels),
                                   nn.Dropout(),
                                   block(input_channels*3),
                                   nn.Dropout(),
                                   block(input_channels*9))
        
        self.second_model = nn.Sequential(nn.Linear(729, 250),
                                         nn.Dropout(),
                                         nn.ReLU(),
                                         nn.Linear(250, 40),
                                         nn.Dropout(),
                                         nn.ReLU(),
                                         nn.Linear(40, 7))
        
        self.quantum_model = Hybrid(qiskit.Aer.get_backend('qasm_simulator'), 1000, np.pi / 2)
        
        self.end_model = nn.Sequential(
            nn.Linear(8,5),
            nn.Sigmoid()
        )
        
    #The forward pass when you call it   
    def forward(self, images):
        pre_proc = self.model(images)
#         print(pre_proc.shape)
        
        formatted = torch.reshape(pre_proc, (images.shape[0], -1))
        
        result = self.second_model(formatted)
        
        result = self.quantum_model(result)
        
        result = result.type(dtype=torch.cuda.FloatTensor)
        result = self.end_model(result.float())
        
        result = result.view(1, -1)
        
        return torch.cat((result, 1-result), -1)

In [ ]:
model = cnnmaker(3)

In [ ]:
model

In [ ]:
#Look at how many parameters
total = 0
for param in model.parameters():
    if param.requires_grad:
        total += param.numel()

print(total)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
model = model.to('cuda')
loss_func = loss_func.to('cuda')

In [ ]:
def accuracy(preds, target):
    correct = (preds == target).float()
    accuracy = correct.sum() / len(correct)
    return accuracy

In [ ]:
def train():
    model.train()
    
    for image, label in dl:
        image, label = image.type(dtype=torch.cuda.FloatTensor), label.type(dtype=torch.cuda.LongTensor)
        optimizer.zero_grad()
        prediction = model(image)
        
        loss = loss_func(prediction, label)
        
        loss.backward()
        
        optimizer.step()
    
    with torch.no_grad():
        total_iter = 0
        total_acc = 0
        for image, label in valid_dl: 
            image, label = image.type(dtype=torch.cuda.FloatTensor), label.type(dtype=torch.cuda.LongTensor)
            prediction = model(image)
            true_pred = prediction.argmax(dim=1, keepdim = True)
            total_acc += true_pred.eq(label.view_as(true_pred)).sum().item()
            total_iter += 1
            if total_iter == 400:
                print("valid",total_acc/total_iter)
                total_iter = 0
                total_acc = 0
                for image, label in dl: 
                    image, label = image.type(dtype=torch.cuda.FloatTensor), label.type(dtype=torch.cuda.LongTensor)
                    prediction = model(image)
                    true_pred = prediction.argmax(dim=1, keepdim = True)
                    total_acc += true_pred.eq(label.view_as(true_pred)).sum().item()
                    total_iter += 1
                    if total_iter == 200:
                        print(total_acc/total_iter)
                        return "done"

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.003)

In [ ]:
for _ in range(4):
    train()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
for _ in range(10):
    train()

In [ ]:
for _ in range(20):
    train()

In [ ]:
for _ in range(20):
    train()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
for _ in range(20):
    train()

In [ ]:
xxx = 0
yyy = 0

for x,y in dl:
    print(x.shape,y.shape)
    xxx = x
    yyy = y
    break

In [ ]:
yyy = yyy.type(dtype=torch.cuda.FloatTensor)
torch.argmax(yyy, dim = 1).float()

In [ ]:
xxx = xxx.type(dtype=torch.cuda.FloatTensor)
prediction = model(xxx)
torch.argmax(prediction, dim = 1).float()


In [ ]:
accuracy(torch.argmax(prediction, dim = 1).float(),torch.argmax(yyy, dim = 1).float())

In [ ]:
with torch.no_grad():
    total_iter = 0
    total_acc = 0
    for image, label in valid_dl: 
        image, label = image.to('cuda', dtype=torch.float), label.to('cuda', dtype=torch.float)
        prediction = model(image)
        true_pred = prediction.argmax(dim=1, keepdim = True)
        total_acc += true_pred.eq(label.view_as(true_pred)).sum().item()
#         print(label)
#         print(true_pred)
        total_iter += 1
        if total_iter == 700:
            print(total_acc/total_iter)
            break

In [ ]:
dds = torchvision.datasets.ImageFolder("../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images", transform_bunch)#, target_transform = target_to_oh)

In [ ]:
distribution = [0] * 5

In [ ]:
distribution

In [ ]:
for x,y in dl:
    distribution[y] = distribution[y] + 1

In [ ]:
distribution

In [ ]:
637+1805+295+193

In [ ]:
distribution[0][0] = distribution[0][0] + 1

In [ ]:
distribution[0] = [2]

In [ ]:
distribution